In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_yelp = pd.read_csv('C:/Users/User/Desktop/DS-Projects/NLP/yelp_labelled.txt', sep='\t',header=None)

In [3]:
data_yelp.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [4]:
# Assign column names
columan_name = ['Review', 'Sentiment']
data_yelp.columns = columan_name

In [5]:
data_yelp.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [6]:
data_yelp.shape

(1000, 2)

In [7]:
data_amazon = pd.read_csv('C:/Users/User/Desktop/DS-Projects/NLP/amazon_cells_labelled.txt',sep='\t',header=None)
data_amazon.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [8]:
data_amazon.columns = columan_name

In [9]:
data_amazon.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [10]:
data_amazon.shape

(1000, 2)

In [11]:
data_imdb = pd.read_csv('C:/Users/User/Desktop/DS-Projects/NLP/imdb_labelled.txt',sep='\t',header=None)
data_imdb.head()

,0,1
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [12]:
data_imdb.columns = columan_name

In [13]:
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [14]:
data_imdb.shape

(748, 2)

In [15]:
data = data_yelp.append([data_amazon, data_imdb],ignore_index=True)

C:\Users\User\AppData\Local\Temp\ipykernel_11988\2635526589.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data_yelp.append([data_amazon, data_imdb],ignore_index=True)


In [16]:
data.shape

(2748, 2)

In [17]:
data['Sentiment'].value_counts()

1    1386
0    1362
Name: Sentiment, dtype: int64

In [18]:
# check for null values
data.isnull().sum()


Review       0
Sentiment    0
dtype: int64

In [19]:
x = data['Review']
y = data['Sentiment']

In [20]:
#Data Cleaning
# here we will remove stopwords, punctuations as well as we will apply lemmatization

import string
punct = string.punctuation

In [21]:
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [22]:
from spacy.lang.en.stop_words import STOP_WORDS

In [23]:
# list of stopwords
stopwords = list(STOP_WORDS)

In [24]:
# creating a function for data cleaning
import spacy 

nlp = spacy.load('en_core_web_sm')

def text_data_cleaning(sentence):
  doc = nlp(sentence)

  tokens = [] # list of tokens
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)
  
  cleaned_tokens = []
  for token in tokens:
    if token not in stopwords and token not in punct:
      cleaned_tokens.append(token)
  return cleaned_tokens

# if root form of that word is not pronoun then it is going to convert that into lower form
# and if that word is a proper noun, then we are directly taking lower form, because there is no lemma for proper noun

In [25]:
text_data_cleaning("Hello all, It's a beautiful day outside there!")
# stopwords and punctuations removed

['hello', 'beautiful', 'day', 'outside']

In [26]:
#Vectorization Feature Engineering (TF-IDF)
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [27]:
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning)

In [34]:
classifier = LinearSVC()
#Splitting the dataset into the Train and Test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [35]:
x_train.shape, x_test.shape

((2198,), (550,))

In [36]:
x_train.head()

2572    An Italian reviewer called this "a small, grea...
526                          And it was way to expensive.
1509    As an earlier review noted, plug in this charg...
144     Nice blanket of moz over top but i feel like t...
2483    The film gives meaning to the phrase, "Never i...
Name: Review, dtype: object

In [37]:
#Fit the x_train and y_train
clf = Pipeline([('tfidf',tfidf), ('clf',classifier)])

In [38]:
clf.fit(x_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x000001CEBA7725F0>)),
                ('clf', LinearSVC())])

In [39]:
#Predict the Test set results
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
y_pred = clf.predict(x_test)


In [40]:
# confusion_matrix
confusion_matrix(y_test, y_pred)

array([[200,  79],
       [ 51, 220]], dtype=int64)

In [41]:
# classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.72      0.75       279
           1       0.74      0.81      0.77       271

    accuracy                           0.76       550
   macro avg       0.77      0.76      0.76       550
weighted avg       0.77      0.76      0.76       550

